In [1]:
import os
import time
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

#from skimage.filters.rank import entropy
import sample_rasters as sr
from rasterio.crs import CRS

from shapely.geometry import box

import planetary_computer as pc

In [2]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = pd.read_csv(os.path.join(os.getcwd(),'temp', 'aoi_naip_itemids.csv'))

csv_name = 'spectral_window_test_set.csv'
all_pts = pd.read_csv(os.path.join(os.getcwd(), csv_name))

# ***************************************************
# ***************************************************

In [3]:
# temporary folder for aux rasters
folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
if os.path.exists(folp) == False:
    os.mkdir(folp)

In [4]:
t0 = time.time()

sampled_pts = []

for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids.itemid[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    # ---------------------------------------
    # find polygons for that NAIP scene
    poly_fp = sr.path_to_polygons(itemids.iloc[i].aoi_name, itemids.iloc[i].year)
    polys = gpd.read_file(poly_fp)
    
    # iterate through polygons in scene
    for j in list(polys.id):
        # ---------------------------------------
        # find points in current polygon
        pts_poly = all_pts.loc[ (all_pts['naip_id'] == itemid) & (all_pts['polygon_id'] == j)]
        if len(pts_poly) !=0:
            crs = CRS.from_string(pts_poly.pts_crs.iloc[0])
            pts_poly_df = sr.geodataframe_from_csv(df = pts_poly, lon_label='x', lat_label='y', crs=crs)
            pts_col = pts_poly_df.to_crs(naip_rast_r.rio.crs).geometry

            # ---------------------------------------
            # create enlarged bounding box

            # (this is ugly, but unfortunately the index and the polygon.id do not match in some files)
            poly_index = polys.index[polys['id']==j].tolist()[0]                

            poly = polys.geometry[poly_index]
            reduce = gpd.GeoDataFrame({'geometry':[box(*poly.bounds)]}, crs=polys.crs)
            reduce = reduce.to_crs(naip_rast_r.rio.crs) 
            poly = reduce.geometry[0]  # poly in scene's crs
            reduce_box = box(*(poly.buffer(6).bounds)) # n must be 2*(entropy disk radius)

            # ---------------------------------------
            # clip NAIP scene
            rast = naip_rast_r.rio.clip_box(*reduce_box.bounds)

            # ---------------------------------------
            # save auxiliary entropy rasters for R,G,B,NIR bands of clipped scene
            band_names = ['r_', 'g_', 'b_', 'nir_']
            tags = ['_entrs']
            window_fps = []
            window_cols = []

            for band_name, band_n in zip(band_names,range(1,5)):
                rast_name = band_name + itemid + '_poly_'+str(j)
                sr.entropy_raster(raster = rast, band=band_n, rast_name=rast_name, n=3, folder_path=folp)

                for tag in tags:
                    window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
                    window_cols.append( band_name.replace('_','')+tag.replace('s',''))

            # ------------------------------
            # make auxiliary NDVI of clipped scene
            red_band = rast.sel(band=1).astype('int16') 
            nir_band = rast.sel(band=4).astype('int16')
            ndvi = ((nir_band - red_band) / (nir_band + red_band)*100)+100

            # make auxiliary NDVI entropy
            band_names.append('ndvi_')
            rast_name = 'ndvi_' + itemid + '_poly_'+str(j)
            sr.entropy_raster(rast_data=ndvi.astype('uint8'),
                              crs=rast.rio.crs, 
                              transf=rast.rio.transform(), 
                              rast_name=rast_name, 
                              n=3, 
                              folder_path=folp)

            for tag in tags:
                window_fps.append(os.path.join(folp, rast_name + tag + '.tif'))        
                window_cols.append( 'ndvi'+tag.replace('s',''))

            # ---------------------------------------
            # sample raster values for points in current polygon
            samples = []
            for fp, col_name in zip(window_fps, window_cols):
                rast_r = rasterio.open(fp)
                sample = sr.sample_raster_from_pts(pts_col, rast_r, [col_name])    
                samples.append(sample)

            # ---------------------------------------
            # Add all derived spectral data to pts dataframe
            new_features = pd.concat(samples, axis = 1)
            pts = pd.concat([pts_poly, new_features.set_index(pts_poly_df.index)], axis=1)                

            # -----------------------------
            # collect all points from each polygon in the scene
            sampled_pts.append(pts)

            # ---------------------------------------
            # delete aux entropy rasters
            for fp in window_fps:
                os.remove(fp)
            
sampled_pts= pd.concat(sampled_pts).sort_index()
        
print(time.time()-t0)

1.9172253608703613


In [5]:
sampled_pts

,x,y,pts_crs,aoi,naip_id,polygon_id,r,r_max,r_min,r_diff,...,min_lidar,min_max_diff,avg_lidar,iceplant,geometry,r_entr,g_entr,b_entr,nir_entr,ndvi_entr
0,734828.437895,3.816920e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,51,120,129,118,11,...,0.0,0.0,0.000000,0.0,POINT (734828.438 3816920.135),3.464658,3.926272,3.505592,3.814372,3.397135
1,734782.017794,3.816923e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,51,120,124,117,7,...,0.0,5.0,3.000000,0.0,POINT (734782.018 3816922.652),3.138177,3.620936,2.434510,3.319043,2.680897
2,733147.107072,3.814899e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,52,88,100,81,19,...,2.0,2.0,3.222222,1.0,POINT (733147.107 3814899.436),3.689902,2.573883,2.762985,2.787020,3.164208
3,734286.276386,3.819001e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,49,122,128,118,10,...,0.0,0.0,0.000000,0.0,POINT (734286.276 3819000.629),3.414040,3.620936,3.358535,2.856633,2.775893
4,732218.875032,3.817980e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,42,97,99,80,19,...,0.0,1.0,0.888889,0.0,POINT (732218.875 3817979.724),4.142295,3.995238,4.418157,3.659537,3.400579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231363,264456.074356,3.812319e+06,epsg:26911,carpinteria,ca_m_3411936_se_11_1_20120505_20120730,45,85,90,72,18,...,NaN,NaN,NaN,0.0,POINT (264456.074 3812319.043),3.952303,3.771437,3.456975,3.909368,3.978333
231364,268707.003692,3.811113e+06,epsg:26911,carpinteria,ca_m_3411936_se_11_1_20120505_20120730,46,80,81,75,6,...,NaN,NaN,NaN,0.0,POINT (268707.004 3811112.780),3.374548,3.952303,3.529384,4.116265,2.813698
231365,268181.709791,3.808520e+06,epsg:26911,carpinteria,ca_m_3411936_se_11_1_20120505_20120730,9,122,141,121,20,...,NaN,NaN,NaN,1.0,POINT (268181.710 3808520.016),4.073330,3.624380,3.952303,3.978333,3.823498
231366,270113.354020,3.807913e+06,epsg:26911,carpinteria,ca_m_3411936_se_11_1_20120505_20120730,72,124,129,123,6,...,NaN,NaN,NaN,0.0,POINT (270113.354 3807912.891),3.302139,3.279552,3.095243,3.331613,2.567696
